In [1]:
from bs4 import BeautifulSoup

In [2]:
# !pip install beautifulsoup4

In [3]:
import urllib
import  bs4

NICK

In [61]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.nike.com/fr/help/a/reduction-pour-etudiants-ue'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Use find to directly select a specific element
    category_element = soup.find('div', class_='card-image-wrapper typography-body-1 css-0')

    if category_element:
        # Find the <p> tag within the selected element
        description = category_element.find('p')

        if description:
            description_text = description.text.strip()
            print(f"Description: {description_text}")
        else:
            print("Paragraph not found within the category element.")
    else:
        print("Category element not found.")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Category element not found.


## Back Market

Scraper toutes les categories et les mettres dans un fichier json

In [36]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://www.backmarket.fr/fr-fr/student-discount'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Select all elements with the specified class
    category_elements = soup.find_all('span', class_='body-1-bold flex-grow')

    # Extract text from the parent elements of the selected spans
    categories_list = [element.find_parent().text.strip() for element in category_elements if element.find_parent()]

    # Print and save the list of categories to a JSON file
    result_data = {"categories": categories_list}

    for category in categories_list:
        print(category)

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


scraping the reduction, the description from back market and put them in save them in market_data.json

In [52]:
import requests
from bs4 import BeautifulSoup
import re
import json

market_name = "Back Market"
url = 'https://www.backmarket.fr/fr-fr/student-discount'

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    market_description_element1 = soup.find('p', class_='body-1-light text-primary-light')

    div_element = soup.find('div', class_='px-8 py-9 md:px-0 md:py-[12rem] mb-9 md:mb-[12rem]')
    if div_element:
        market_description_element2 = div_element.find('p')
        if market_description_element2:
            market_description2 = market_description_element2.text.strip()
        else:
            market_description2 = None
    else:
        market_description2 = None

    if market_description_element1:
        market_description = market_description_element1.text.strip()
        match = re.search(r'(-?\d+€)', market_description)
        if match:
            reduction = match.group(1)
        else:
            reduction = None
    else:
        market_description = None
        reduction = None

    # Create a dictionary with the extracted data
    market_data = {
        "market_name": market_name,
        "description1": market_description,
        "reduction": reduction,
        "description2": market_description2
    }

    # Save the data to a JSON file
    with open('market_data.json', 'w', encoding='utf-8') as json_file:
        json.dump(market_data, json_file, indent=2, ensure_ascii=False)

    print("Data saved to market_data.json.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Data saved to market_data.json.


##DELL

market name , reduction and the description

market categories

In [55]:
import json
import requests
from bs4 import BeautifulSoup

# Function to save data to a JSON file
def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=2, ensure_ascii=False)

# First part of the code for Dell student offers
url_students = 'https://www.dell.com/fr-fr/lp/students'
response_students = requests.get(url_students)

if response_students.status_code == 200:
    soup_students = BeautifulSoup(response_students.text, 'html.parser')
    market_divs_students = soup_students.find_all('div', class_='rwp-contentlayout')

    market_data_list_students = []
    first_name_encountered = False

    for i in market_divs_students:
        market_description_element = i.find('p', class_='rwp-contentlayout-item__description')

        if market_description_element:

            if not first_name_encountered:
                market_name = market_description_element.text.split()[0]
                print(f"Name: {market_name}")
                first_name_encountered = True
            else:
                # If the first name has been encountered, skip processing additional names
                print("Additional names skipped")

        if market_description_element:
            market_description = market_description_element.text.strip()
            print(f"Description: {market_description}")
            match = re.search(r'(\d+\s*%)', market_description)

            if match:
              reduction = match.group(1)
              print(f"Reduction: {reduction}")
            else:
              print("No percentage reduction found.")
        else:
            print("Description not found")


        # Append data to the list
        market_data_list_students.append({
            "name": market_name,
            "description": market_description,
            "reduction":reduction
        })

else:
    print(f"Failed to retrieve the page. Status code: {response_students.status_code}")

# Second part of the code for Dell PC deals
url_pc_deals = 'https://www.dell.com/fr-fr/shop/deals/top-pc-deals'
response_pc_deals = requests.get(url_pc_deals)

if response_pc_deals.status_code == 200:
    soup_pc_deals = BeautifulSoup(response_pc_deals.text, 'html.parser')
    category_items = soup_pc_deals.select('ul.sub-nav li.child-nav')

    categories_data = {}

    for category_item in category_items:
        main_category_name = category_item.select_one('button.mh-menuItem').text.strip()
        subcategory_buttons = category_item.select('li a.mh-menuItem')
        subcategories = [button.text.strip() for button in subcategory_buttons]
        categories_data[main_category_name] = subcategories

    # Combine both sets of data into a single dictionary
    combined_data = {
        "dell_student_offers": market_data_list_students,
        "dell_pc_deals": categories_data
    }

    # Save the combined data to a JSON file
    save_to_json(combined_data, 'combined_dell_data.json')

    print("Categories and Subcategories:")
    for main_category, subcategories in categories_data.items():
        print(f"Main Category: {main_category}")
        print(f"Subcategories: {', '.join(subcategories)}")
        print("---")
else:
    print(f"Failed to retrieve the page. Status code: {response_pc_deals.status_code}")


Name: DELL
Description: DELL TECHNOLOGIES POUR LES ÉTUDIANTSParé pour l’éducation.Préparé pour la vie.
No percentage reduction found.
Additional names skipped
Description: Remises exclusives pour les étudiantsObtenez jusqu’à 20 % de réduction sur les ordinateurs et les produits électroniques les mieux notés grâce à des bons de réduction exclusifs pour les étudiants. Si vous ne disposez pas encore de votre adresse e-mail académique, veuillez envoyer un e-mail à dell_advantage_student@dell.com pour demander votre bon de réduction.
Reduction: 20 %
Additional names skipped
Description: Solutions de recyclageLe recyclage de vos produits technologiques obsolètes constitue une solution responsable pour nous tous.
No percentage reduction found.
Additional names skipped
Description: Pour moins de déchetsNotre engagement en faveur d’une économie circulaire inclut l’exploitation de la technologie et des partenariats pour repenser, reconcevoir, réutiliser et recycler pour un avenir meilleur.
No pe

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
